In [ ]:
# Python - 3.13.7
# Dependencies - langchain-community, langchain-text-splitters, langchain-milvus, langchain-huggingface, langchain-qdrant, chromadb, faiss-cpu, pypdf, langchain, sentence-transformers

In [ ]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_milvus import Milvus
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores import Chroma

In [2]:
folder_path = "./Netsuite_pdfs/"

In [ ]:
# Parsing pdfs, not needed if parsed_docs.json present

# loader = DirectoryLoader(
#     path=folder_path,
#     glob='*.pdf',
#     loader_cls=PyPDFLoader
# )

# docs = loader.lazy_load()

# pages = []
# async for page in loader.alazy_load():
#     pages.append(page)

In [ ]:
# Functions for saving and loading parsed documents

import json


def save_documents(docs, filename="docs.json"):
    data = [{"page_content": d.page_content, "metadata": d.metadata}
            for d in docs]
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)


def load_documents(filename="docs.json"):
    from langchain.schema import Document
    import json
    with open(filename, "r", encoding="utf-8") as f:
        data = json.load(f)
    return [Document(page_content=d["page_content"], metadata=d["metadata"]) for d in data]

In [ ]:
# not needed if parsed_docs.json present

# save_documents(pages, "parsed_docs.json")

In [4]:
pages = load_documents("parsed_docs.json")

In [8]:
len(pages)

32388

In [ ]:
# Chunk size set due to sentence-transformers/all-mpnet-base-v2 constraints

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1400, chunk_overlap=150)
texts = text_splitter.split_documents(pages)

In [6]:
len(texts)

71702

In [2]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2")

c:\Users\IDEAPAD\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Milvus DB

In [ ]:
URI = "./milvus_example.db"

milvus_vector_store = Milvus(
    embedding_function=embeddings,
    connection_args={"uri": URI},
    index_params={"index_type": "FLAT", "metric_type": "L2"},
)

In [ ]:
milvus_vector_store.add_documents(texts)

In [ ]:
milvus_retriever = milvus_vector_store.as_retriever(
    search_type="mmr", search_kwargs={"k": 10, "lambda_mult": 0.8})

In [ ]:
milvus_retriever.invoke(
    "What are the standard and specialized NetSuite centers?")

## Qdrant

In [ ]:
# No need to run if db is already present

client = QdrantClient(path="./tmp/langchain_qdrant")

client.create_collection(
    collection_name="demo_collection",
    vectors_config=VectorParams(size=768, distance=Distance.COSINE),
)

qdrant_vector_store = QdrantVectorStore(
    client=client,
    collection_name="demo_collection",
    embedding=embeddings,
)

qdrant_vector_store.add_documents(texts)

True

In [ ]:
# Loading saved database

# client = QdrantClient(path="./tmp/langchain_qdrant")


# qdrant_vector_store = QdrantVectorStore(
#     client=client,
#     collection_name="demo_collection",
#     embedding=embeddings,
# )

In [ ]:
qdrant_retriever = qdrant_vector_store.as_retriever(
    search_type="mmr", search_kwargs={"k": 10, "lambda_mult": 0.8})


In [ ]:
qdrant_retriever.invoke(
    "How do I set up commission calculations for sales reps?")

## Faiss

In [ ]:
faiss_vector_store = FAISS.from_documents(texts, embeddings)

In [ ]:
faiss_vector_store.save_local("faiss_index")

In [ ]:
faiss_retriever = faiss_vector_store.as_retriever(
    search_type="mmr", search_kwargs={"k": 10, "lambda_mult": 0.8})

In [ ]:
faiss_retriever.invoke(
    "How do I set up commission calculations for sales reps?")

## Chroma

In [ ]:
chroma_vector_store = Chroma.from_documents(
    documents=texts,
    embedding=embeddings,
    persist_directory="./chroma_db"
)

In [ ]:
chroma_retriever = chroma_vector_store.as_retriever(
    search_type="mmr", search_kwargs={"k": 10, "lambda_mult": 0.8})

In [ ]:
chroma_retriever.invoke(
    "How do I set up commission calculations for sales reps?")